## Groubi基本操作

&emsp;&emsp;安装：
- conda install -c gurobi gurobi

## Gurobi建模过程

1. `Problem Instance`:待优化问题。
2. `Model Generator`: 将数据组合成模型, 产生计算机模型对象。
3. `Model Instance`: 存在于内存的一个完整数学模型。
4. `Gurobi Optimizer`: `Gurobi`优化求解。
5. `Solution Retrieval`: 根据需要读取优化结果。

&emsp;&emsp;循环往复，直到获得满意结果。

## Gurobi建模基本概念

1. Parameter(参数)控制优化器的行为，需要在优化启动前设置。
2. Attributes(属性)控制模型(包括模型、变量、约束、目标等对象)的特性。

&emsp;&emsp;例如：

&emsp;&emsp;模型 Model｜Sense

&emsp;&emsp;变量 LB｜UB

&emsp;&emsp;约束 RHS

&emsp;&emsp;`Enviornment`是包含模型和全局参数的一个容器，也是许可控制的节点。

&emsp;&emsp;在建模过程中，经常要对带下标数据做挑选，不同下标的数据进行组合，这样面临着两个处理方法：

1. 全部循环，多维下表意味着多重循环+if条件，这样的处理方法没有效率。
2. 采用特殊的`Gurobi`扩展对象`TupleList`和`TupleDict`。

### Tuplelist

In [1]:
from gurobipy import *

In [2]:
Cities = [('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'D'), ('C', 'D')]
Routes = tuplelist(Cities)
Routes

<gurobi.tuplelist (5 tuples, 2 values each):
 ( A , B )
 ( A , C )
 ( B , C )
 ( B , D )
 ( C , D )
>

In [3]:
print(Routes.select('A', '*'))  # tuplelist增加了快速筛选select功能。

<gurobi.tuplelist (2 tuples, 2 values each):
 ( A , B )
 ( A , C )
>


如果采用Python本身的语法的话，就是如下语句：

In [4]:
Result = []
for i, j in Cities:
    if i == "A":
        Result.append((i, j))
print(Result)

[('A', 'B'), ('A', 'C')]


### Tupledict

&emsp;&emsp;键值为`tuple`(元祖)，可以使用`select`，`sum`，`prod`函数。用于变量和约束(后面案例中体现)。

### Multidict

Multidict创建tuplelist和tupledict的便捷方法。

In [5]:
cities, supply, demand = multidict({'A':[100, 20],
                                    'B':[150, 50],
                                    'C':[20, 300],
                                    'D':[10, 200]})

In [6]:
cities

['A', 'B', 'C', 'D']

In [7]:
supply

{'A': 100, 'B': 150, 'C': 20, 'D': 10}

In [8]:
demand

{'A': 20, 'B': 50, 'C': 300, 'D': 200}

## Guribo建模过程

&emsp;&emsp;建模过程中建议是创建变量的时候就创建所有的变量，设置目标函数，创建所有的约束，优化求解：

1. addVar(), addVars
2. setobjective()
3. addConstr(), addConstrs()
4. optimize()

## Gurobi建模举例1，优化目标函数

$$
\begin{aligned}
\max & \ \ x+y+2 z \\
\text { s.t. } & \ \ x+2 y+3 z \leq 4 \\
& x+y \geq 1 \\
& x, y, z \in\{0,1\}
\end{aligned}
$$

In [9]:
from gurobipy import *
try:
    # 创建一个新的模型
    m = Model("mip1")
    
    # 创建变量
    x = m.addVar(vtype=GRB.BINARY, name="x")  # GRB.BINARY表示01变量
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")
    
    # 设置目标函数
    m.setObjective(x + y + 2*z, GRB.MAXIMIZE)  # 方向表示最大化
    
    # 添加约束
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")  # 给约束条件添加一个名字，c0，方便之后修改的时候快速定位到某一个约束。
    m.addConstr(x + y >= 1, "c1")  # 添加另外一个约束
    
    m.optimize()
    
    # 获取所有的变量
    for v in m.getVars():
        print("{} {}".format(v.varName, v.x))
    
    print("obj: {}".format(m.objVal))
except GurobiError as e:
    print("Error code " + str(e.errno) + ": " + str(e))
except AttributeError:
    print("Encounter an attribute error")
    

Restricted license - for non-production use only - expires 2022-01-13
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1.0
y 0.0
z 1.0
obj: 3.0


## Gurobi建模举例2，营养配方模型

&emsp;&emsp;人体需要四种营养：category: calories, protein, fat, sodium

&emsp;&emsp;食物来源：food=hamburger, chicken, hot dog, fries, macaroni, pizza, salad, milk, icecream

&emsp;&emsp;营养吸收每天有上限和下限，单位重量食物价格不同，单位重量食物所含营养成分不同。

求达到足够营养花费的代价最小。

In [10]:
from gurobipy import *

&emsp;&emsp;设置营养类型，营养的上线和营养的下限：

In [11]:
categories, minNutrition, maxNutrition = multidict({
    'calories': [1800, 2200],
    'protein': [91, GRB.INFINITY],
    'fat': [0, 65],
    'sodium': [0, 1779]})
print("categories", categories)
print("minNutrition", minNutrition)

categories ['calories', 'protein', 'fat', 'sodium']
minNutrition {'calories': 1800, 'protein': 91, 'fat': 0, 'sodium': 0}


In [12]:
foods, cost = multidict({
    'hamburger': 2.49,
    'chicken': 2.89,
    'hot dog': 1.50,
    'fries': 1.89,
    'macaroni': 2.09,
    'pizza': 1.99,
    'salad': 2.49,
    'milk': 0.89,
    'ice cream': 1.59})

In [13]:
nutritionValues = {
    ('hamburger', 'calories'): 410,
    ('hamburger', 'protein'): 24,
    ('hamburger', 'fat'): 26,
    ('hamburger', 'sodium'): 730,
    ('chicken', 'calories'): 420,
    ('chicken', 'protein'): 32,
    ('chicken', 'fat'): 10,
    ('chicken', 'sodium'): 1190,
    ('hot dog', 'calories'): 560,
    ('hot dog', 'protein'): 20,
    ('hot dog', 'fat'): 32,
    ('hot dog', 'sodium'): 1800,
    ('fries', 'calories'): 380,
    ('fries', 'protein'): 4,
    ('fries', 'fat'): 19,
    ('fries', 'sodium'): 270,
    ('macaroni', 'calories'): 320,
    ('macaroni', 'protein'): 12,
    ('macaroni', 'fat'): 10,
    ('macaroni', 'sodium'): 930,
    ('pizza', 'calories'): 320,
    ('pizza', 'protein'): 15,
    ('pizza', 'fat'): 12,
    ('pizza', 'sodium'): 820,
    ('salad', 'calories'): 320,
    ('salad', 'protein'): 31,
    ('salad', 'fat'): 12,
    ('salad', 'sodium'): 1230,
    ('milk', 'calories'): 100,
    ('milk', 'protein'): 8,
    ('milk', 'fat'): 2.5,
    ('milk', 'sodium'): 125,
    ('ice cream', 'calories'): 330,
    ('ice cream', 'protein'): 8,
    ('ice cream', 'fat'): 10,
    ('ice cream', 'sodium'): 180}

In [14]:
## 定义模型
m = Model("diet")

# 从食物中去创建决策变量
buy = m.addVars(foods, name="buy")  # 每个变量名字的开始字符串都是buy。
print(buy)

{'hamburger': <gurobi.Var *Awaiting Model Update*>, 'chicken': <gurobi.Var *Awaiting Model Update*>, 'hot dog': <gurobi.Var *Awaiting Model Update*>, 'fries': <gurobi.Var *Awaiting Model Update*>, 'macaroni': <gurobi.Var *Awaiting Model Update*>, 'pizza': <gurobi.Var *Awaiting Model Update*>, 'salad': <gurobi.Var *Awaiting Model Update*>, 'milk': <gurobi.Var *Awaiting Model Update*>, 'ice cream': <gurobi.Var *Awaiting Model Update*>}


In [15]:
# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# 以下这种写法是等价的。
# buy = {}
# for f in foods:
#   buy[f] = m.addVar(name=f)

&emsp;&emsp;我们要定义的目标函数就是最小化每个食物需要购买的量，与食物的价格的乘积：

In [16]:
# Using looping constructs, the preceding statement would be:
# m.setObjective(sum(buy[f]*cost[f] for f in foods), GRB.MINIMIZE)

In [17]:
# The objective is to minimize the costs
m.setObjective(buy.prod(cost), GRB.MINIMIZE)

&emsp;&emsp;约束条件就是每一个营养成分来自所有营养成分的组合，需要保持在每一天的营养的上界和下界之间。

In [18]:
# Using looping constructs, the preceding statement would be:
# 循环遍历每种营养，所有食物的这种营养总和需要在这个上下界中。
# for c in categories:
#     m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods),
#                minNutrition[c], maxNutrition[c], c)

In [19]:
# Nutrition constraints
m.addConstrs((quicksum(nutritionValues[f, c] * buy[f] for f in foods)
              == [minNutrition[c], maxNutrition[c]]
              for c in categories), name='Nutrition')

{'calories': <gurobi.Constr *Awaiting Model Update*>,
 'protein': <gurobi.Constr *Awaiting Model Update*>,
 'fat': <gurobi.Constr *Awaiting Model Update*>,
 'sodium': <gurobi.Constr *Awaiting Model Update*>}

In [20]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0x33ddb849
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.472500e+02   0.000000e+00      0s
       4    1.1828861e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective  1.182886111e+01


In [21]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nCost: %g' % m.objVal)
        print('\nBuy:')
        for f in foods:
            if buy[f].x > 0.0001:
                print('%s %g' % (f, buy[f].x))
    else:
        print('No solution')

In [22]:
printSolution()


Cost: 11.8289

Buy:
hamburger 0.604514
milk 6.97014
ice cream 2.59132


数据量比较大的时候，可以直接从Excel，ODBC数据库等读入，然后用multidict, list.append, list[n]=xxx, dict[k1, ke,,]=value等方式赋值。